In [1]:
from pathlib import Path
import json
from collections import defaultdict

In [2]:
import cv2
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image, ImageOps
import torch
import torchvision
from torch.utils.data import Dataset
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.io import read_image
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Lambda

In [3]:
torchvision.__version__

'0.10.0+cu111'

In [4]:
torch.__version__

'1.9.0+cu111'

In [5]:
# data_dir = Path(r"C:\datasets\pubtabnet")
data_dir = Path(r"C:\Users\stans\Documents\Projects\Datasets\pubtabnet.tar\pubtabnet\ptnLite")
train_dir = data_dir / "Images"

In [6]:
publaynet_images = list(train_dir.glob("*.png"))
with open(data_dir/'targets.json','r') as f:
    publaynet_targets = json.load(f)

In [36]:
COLORS = {
    1:(0,0,0.8),
    2:(0.8,0,0),
}

In [71]:
def pad_input(img, shape):
    arr = np.zeros(shape)
    w,h = img.size
    arr[:h, :w, 0] = np.array(img)/255
    arr[:h, :w, 1] = np.array(img)/255
    arr[:h, :w, 2] = np.array(img)/255
    return torch.as_tensor(arr.T).type(torch.cuda.FloatTensor)

def get_data(img_path):
    image = Image.open(img_path)
    labels, bboxes = publaynet_targets[img_path.name]
    image = pad_input(image, (256, 256, 3))
    return image, {
        'boxes': torch.as_tensor(bboxes),
        'labels': torch.as_tensor(labels,dtype=torch.int64),
    }

def load_batch(paths):
    X = []
    y = []
    for img_path in paths:
        i,l = get_data(img_path)
        X.append(i)
        y.append(l)
    return X,y

def test_resizer(i):
    im, target = get_data(publaynet_images[i])
    print(f"image: {type(im)} {im.shape} {im.dtype}")
    print(f"labels: {type(target['labels'])} {target['labels'].shape} {target['labels'].dtype}")
    print(f"boxes: {type(target['boxes'])} {target['boxes'].shape} {target['boxes'].dtype}")
    im = np.array(im.T)
    for label, box in zip(target['labels'], target['boxes']):
        x1,y1,x2,y2 = box
        c = COLORS[int(label)]
        im[y1:y2,x1] = c
        im[y1:y2,x2] = c
        im[y1,x1:x2] = c
        im[y2,x1:x2] = c
    plt.imshow(im)

In [73]:
# test_resizer(1)

In [3]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
    pretrained=True,
)
# model.cuda()

In [ ]:
model.eval()
image.open()

In [75]:
def train_loop(files, batch_size, model):
    size = len(files)
    for batch in range(size//batch_size):
        im_paths = files[batch*batch_size : (batch+1)*batch_size]
        X,y = load_batch(im_paths)
        # Compute prediction and loss
        output = model(X, y)
        print(torch.cuda.max_memory_allocated())
        if batch % 100 == 0:
            current = (1+batch)*batch_size
            print(f"{current}/{size}".center(40,'-'))
            for k,v in output.items():
                print(f"{k}{' '*(20-len(k))}{v}")

In [76]:
batch_size = 8
epochs = 5

In [77]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(publaynet_images[:100000], batch_size, model)
print("Done!")

Epoch 1
-------------------------------


RuntimeError: CUDA out of memory. Tried to allocate 314.00 MiB (GPU 0; 3.00 GiB total capacity; 1.07 GiB already allocated; 304.41 MiB free; 1.32 GiB reserved in total by PyTorch)

In [43]:
torch.cuda.max_memory_allocated()

0